In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 
import os
# import seaborn as sns
# import scipy.stats as stats
import unittest

In [59]:
df_battery = pd.read_csv('Battery_Dataset.csv')

In [60]:
import csv
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold 
import math

import sklearn
import statistics as stcs
import scipy.stats as stats


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,scale,MinMaxScaler
from scipy.stats import uniform,expon
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error

In [61]:
#One hot encoding (alphebetical order)
ohe = OneHotEncoder()
ACE = df_battery.loc[:,['anode','cathode','electrolyte']]
ACE = ohe.fit_transform(ACE)
ACE = pd.DataFrame(ACE.toarray())

ACE_col_name = ['A1','C1','C2','C3','E1','E2','E3']
for i in range(len(ACE.columns)):
    ACE = ACE.rename({ACE.columns[i]: ACE_col_name[i]}, axis=1) 
    
df_battery = pd.concat([ACE, df_battery],axis=1)

In [62]:
data = df_battery

np.random.seed(66)
def data_split (data, test_ratio):
    df = pd.DataFrame(data)
    total_row = df.shape[0]

    test_row = round(total_row *test_ratio)

    train_row = total_row -test_row
    
    indices =np.random.permutation(total_row)
    train_indx, test_idx =indices[:train_row], indices[train_row:]
    train,test = df.iloc[train_indx,:], df.iloc[test_idx,:]
    X_test = test[['A1','C1','C2','C3','E1','E2','E3','Cycle','temperature','discharge_crate']]
#    y_test = test[['Charge_Capacity (Ah)']]
    y_test = test[['Discharge_Capacity (Ah)']]
#    y_test = test[['Charge_Energy (Wh)']]
#    y_test = test[['Discharge_Energy (Wh)']]
#    y_test = test[['Coulombic_Efficiency (%)']]
#    y_test = test[['Energy_Efficiency (%)']]
    X_train = train[['A1','C1','C2','C3','E1','E2','E3','Cycle','temperature','discharge_crate']]
#    y_train = train[['Charge_Capacity (Ah)']]
    y_train = train[['Discharge_Capacity (Ah)']]
#    y_train = train[['Charge_Energy (Wh)']]
#    y_train = train[['Discharge_Energy (Wh)']]
#    y_train = train[['Coulombic_Efficiency (%)']]
#    y_train = train[['Energy_Efficiency (%)']]
    return train,test,X_train,y_train,X_test,y_test
    
train,test,X_train,y_train,X_test,y_test = data_split(data,0.2)


In [63]:
#k-fold cross validation
kf = KFold(n_splits=10,random_state =66,shuffle= True)
#print(X_test,y_test,X_train,y_train)

In [64]:
#Data scaling, algorithms that use distance measures are affected by the scale of numerical input variables
#Standardization assumes that your observations fit a Guassian distribution with well-behaved mean and standard deviation
scaler = StandardScaler(with_mean=True,with_std=True)
print(scaler.fit(X_test))
print(scaler.mean_)

X_test = pd.DataFrame(scaler.transform(X_test))

print(scaler.fit(X_train))
print(scaler.mean_)
X_train = pd.DataFrame(scaler.transform(X_train))

StandardScaler()
[  1.         0.296875   0.34375    0.359375   0.296875   0.359375
   0.34375  292.1875    25.3125     1.4375  ]
StandardScaler()
[1.00000000e+00 3.94531250e-01 2.65625000e-01 3.39843750e-01
 3.94531250e-01 3.39843750e-01 2.65625000e-01 2.70703125e+02
 2.53125000e+01 1.45703125e+00]


In [65]:
#tuning hyperparameters
rand_para = {'n_neighbors':range(1,201)}

svm_model =KNeighborsRegressor()
n_iteration = 200
Scoring = ['explained_variance','neg_mean_squared_error',"r2"]
random_search = RandomizedSearchCV(svm_model,
                                   param_distributions=rand_para,
                                   cv =5,
                                   scoring=Scoring,
                                   error_score=0,
                                   random_state=66,
                                   return_train_score=True,
                                   n_jobs=-1,
                                   n_iter=n_iteration,
                                   verbose=10,
                                   refit=False)



random_search.fit(X_train,y_train)

#y_predict=random_search.predict(X_test)
#math.sqrt(stcs.mean((y_predict-y_test)**2))

print('Best hyperparameters: ', random_search.best_params_)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 4/5; 1/200] START n_neighbors=1.............................................
[CV 4/5; 1/200] END n_neighbors=1; explained_variance: (train=0.599, test=0.616) neg_mean_squared_error: (train=-0.305, test=-0.337) r2: (train=0.592, test=0.613) total time=   0.0s
[CV 4/5; 3/200] START n_neighbors=3.............................................
[CV 4/5; 3/200] END n_neighbors=3; explained_variance: (train=0.715, test=0.791) neg_mean_squared_error: (train=-0.214, test=-0.182) r2: (train=0.715, test=0.791) total time=   0.0s
[CV 2/5; 4/200] START n_neighbors=4.............................................
[CV 2/5; 4/200] END n_neighbors=4; explained_variance: (train=0.723, test=0.611) neg_mean_squared_error: (train=-0.218, test=-0.283) r2: (train=0.723, test=0.609) total time=   0.0s
[CV 2/5; 5/200] START n_neighbors=5.............................................
[CV 2/5; 5/200] END n_neighbors=5; explained_variance: (train=0.72

[CV 4/5; 41/200] END n_neighbors=41; explained_variance: (train=0.546, test=0.718) neg_mean_squared_error: (train=-0.340, test=-0.245) r2: (train=0.546, test=0.718) total time=   0.0s
[CV 5/5; 41/200] START n_neighbors=41...........................................
[CV 5/5; 41/200] END n_neighbors=41; explained_variance: (train=0.588, test=0.556) neg_mean_squared_error: (train=-0.316, test=-0.355) r2: (train=0.587, test=0.555) total time=   0.0s
[CV 1/5; 42/200] START n_neighbors=42...........................................
[CV 1/5; 42/200] END n_neighbors=42; explained_variance: (train=0.597, test=0.535) neg_mean_squared_error: (train=-0.326, test=-0.316) r2: (train=0.597, test=0.506) total time=   0.0s
[CV 2/5; 42/200] START n_neighbors=42...........................................
[CV 2/5; 42/200] END n_neighbors=42; explained_variance: (train=0.602, test=0.472) neg_mean_squared_error: (train=-0.313, test=-0.388) r2: (train=0.602, test=0.463) total time=   0.0s
[CV 3/5; 42/200] STAR

[CV 2/5; 94/200] START n_neighbors=94...........................................
[CV 2/5; 94/200] END n_neighbors=94; explained_variance: (train=0.526, test=0.452) neg_mean_squared_error: (train=-0.392, test=-0.406) r2: (train=0.501, test=0.438) total time=   0.0s
[CV 3/5; 94/200] START n_neighbors=94...........................................
[CV 3/5; 94/200] END n_neighbors=94; explained_variance: (train=0.519, test=0.455) neg_mean_squared_error: (train=-0.388, test=-0.457) r2: (train=0.490, test=0.448) total time=   0.0s
[CV 4/5; 94/200] START n_neighbors=94...........................................
[CV 4/5; 94/200] END n_neighbors=94; explained_variance: (train=0.463, test=0.567) neg_mean_squared_error: (train=-0.412, test=-0.384) r2: (train=0.450, test=0.559) total time=   0.0s
[CV 5/5; 94/200] START n_neighbors=94...........................................
[CV 5/5; 94/200] END n_neighbors=94; explained_variance: (train=0.515, test=0.494) neg_mean_squared_error: (train=-0.394, te

[CV 5/5; 149/200] END n_neighbors=149; explained_variance: (train=0.212, test=0.191) neg_mean_squared_error: (train=-0.606, test=-0.664) r2: (train=0.209, test=0.167) total time=   0.0s
[CV 1/5; 150/200] START n_neighbors=150.........................................
[CV 1/5; 150/200] END n_neighbors=150; explained_variance: (train=0.177, test=0.212) neg_mean_squared_error: (train=-0.666, test=-0.511) r2: (train=0.176, test=0.201) total time=   0.0s
[CV 2/5; 150/200] START n_neighbors=150.........................................
[CV 2/5; 150/200] END n_neighbors=150; explained_variance: (train=0.195, test=0.189) neg_mean_squared_error: (train=-0.635, test=-0.587) r2: (train=0.193, test=0.188) total time=   0.0s
[CV 3/5; 150/200] START n_neighbors=150.........................................
[CV 3/5; 150/200] END n_neighbors=150; explained_variance: (train=0.192, test=0.180) neg_mean_squared_error: (train=-0.617, test=-0.679) r2: (train=0.188, test=0.180) total time=   0.0s
[CV 4/5; 150/

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [ ]:
#Train-error
np.random.seed(66)

KNN_model_train =KNeighborsRegressor(n_neighbors=10)

train_results =[]
train_results_name =['Experimental','Predicted ','RMSE']


for train_index, test_index in kf.split(X_train):
    #print("Train:", train_index, "Validation:", test_index)
    X_training, X_validate = X_train.iloc[train_index], X_train.iloc[test_index]
    y_training, y_validate = y_train.iloc[train_index], y_train.iloc[test_index]
    np.random.seed(66)
    KNN_model_train.fit(X_training,y_training)
    y_train_predicted = KNN_model_train.predict(X_validate)
    number_validate =X_validate.shape[0]
    #squared_deviations=(y_train_predicted-y_validate)**2
    #RMSE=math.sqrt(squared_deviations.sum()/number_validate)
    #RMSE = math.sqrt(stcs.mean((y_train_predicted-y_validate)**2))
    mse = mean_squared_error(y_train_predicted, y_validate)
    RMSE = np.sqrt(mse)
    train_results.append([y_validate,y_train_predicted,RMSE])

In [ ]:
Train_results =pd.DataFrame (train_results,columns=train_results_name)
np.average(Train_results['RMSE'])

In [ ]:
#Test-set Validation
KNN_model =KNeighborsRegressor(n_neighbors=10)
np.random.seed(66)
KNN_model.fit(X_train,y_train)
y_predict=KNN_model.predict(X_test)
#print(y_predict)

In [57]:
number_test = X_test.shape[0]
mse = mean_squared_error(y_predict, y_test)
RMSE = np.sqrt(mse)
RMSE
# squared_deviations=(y_predict-y_test)**2
# math.sqrt(squared_deviations.sum()/number_test)

0.5294763353422889

In [15]:
"""
Train_results.iloc[0,0].reset_index(drop=True)
"""

'\nTrain_results.iloc[0,0].reset_index(drop=True)\n'

In [16]:
"""
pd.DataFrame(Train_results.iloc[i,1])
"""

'\npd.DataFrame(Train_results.iloc[i,1])\n'

In [17]:
"""
#predict_results.drop(labels=None)
Experimental =[]
Predicted =[]

for i in range(Train_results.shape[0]):
    Ex = Train_results.iloc[i,0].reset_index(drop=True)
    Pred = pd.DataFrame(Train_results.iloc[i,1])
    a = pd.concat([Ex,Pred],ignore_index=True, axis =1)
    """

'\n#predict_results.drop(labels=None)\nExperimental =[]\nPredicted =[]\n\nfor i in range(Train_results.shape[0]):\n    Ex = Train_results.iloc[i,0].reset_index(drop=True)\n    Pred = pd.DataFrame(Train_results.iloc[i,1])\n    a = pd.concat([Ex,Pred],ignore_index=True, axis =1)\n    '

In [18]:
"""
#predict_results.drop(labels=None)
Experimental =[]
Predicted =[]

for i in range(Train_results.shape[0]):
    Ex=Train_results.iloc[i,0].to_string(index=False)
    Pred = pd.DataFrame(Train_results.iloc[i,1]).to_string(index_names=False,index=False,header=False)
    Ex_num = Ex.split("\n ")
    Pre = Pred.split("\n ")
    a = np.array([float(i) for i in Ex_num])
    b = np.array([float(c) for c in Pre])
    Experimental.extend(a)
    Predicted.extend(b)
"""

'\n#predict_results.drop(labels=None)\nExperimental =[]\nPredicted =[]\n\nfor i in range(Train_results.shape[0]):\n    Ex=Train_results.iloc[i,0].to_string(index=False)\n    Pred = pd.DataFrame(Train_results.iloc[i,1]).to_string(index_names=False,index=False,header=False)\n    Ex_num = Ex.split("\n ")\n    Pre = Pred.split("\n ")\n    a = np.array([float(i) for i in Ex_num])\n    b = np.array([float(c) for c in Pre])\n    Experimental.extend(a)\n    Predicted.extend(b)\n'

In [19]:
"""
Ex= pd.DataFrame(Experimental)

Pred = pd.DataFrame(Predicted)

results =pd.concat([Ex,Pred],axis=1,sort=False)
"""

'\nEx= pd.DataFrame(Experimental)\n\nPred = pd.DataFrame(Predicted)\n\nresults =pd.concat([Ex,Pred],axis=1,sort=False)\n'

In [20]:
"""
results.to_csv('5NN-EC-TRAIN-EXPERIMENT-PREDICTED.csv')
"""

"\nresults.to_csv('5NN-EC-TRAIN-EXPERIMENT-PREDICTED.csv')\n"

In [21]:
"""
Pre_y_test = pd.DataFrame(y_predict,index=None)
"""

'\nPre_y_test = pd.DataFrame(y_predict,index=None)\n'

In [22]:
"""
Ex_y_test =pd.DataFrame(y_test)
Ex_y_test.reset_index(drop=True, inplace=True)
"""

'\nEx_y_test =pd.DataFrame(y_test)\nEx_y_test.reset_index(drop=True, inplace=True)\n'

In [23]:
"""
results_test =pd.concat([Ex_y_test,Pre_y_test],axis=1,join='inner')
"""

"\nresults_test =pd.concat([Ex_y_test,Pre_y_test],axis=1,join='inner')\n"

In [24]:
"""
results_test.to_csv('KNN-EC-TEST--EXPERIMENT-PREDICTED.csv')
"""

"\nresults_test.to_csv('KNN-EC-TEST--EXPERIMENT-PREDICTED.csv')\n"

In [25]:
"""
import numpy
import matplotlib.axes as ax
import matplotlib.pyplot as plt

x= y_test
y=y_predict 

plt.axis([0,30,0,30])
plt.scatter(y=results.iloc[:,0],x=results.iloc[:,0],c='deepskyblue',label='Training')
plt.scatter(x,y, c='blue',label='Test')
plt.plot([50,300],[50,300],label ='True line',c='red')
plt.legend(loc='lower right')
#plt.title(label='SVM-EC-Results-Plot')

plt.xlabel(r'Experimental Discharge Capacity (Ah)')

#$50^{th}$ Cycle
plt.ylabel(r'Prediscted $50^{th}$ Cycle Discharge Capacity (mAh/g)')
plt.text(60,230,r'$RMSE_{train}=Ah$'+'\n'+r'$RMSE_{test}=Ah$' +"\n"+ '$R^2_{test}=$')

plt.savefig('KNN-Predicted-vs-test.png',dpi=600)
plt.show()
"""

'\nimport numpy\nimport matplotlib.axes as ax\nimport matplotlib.pyplot as plt\n\nx= y_test\ny=y_predict \n\nplt.axis([0,30,0,30])\nplt.scatter(y=results.iloc[:,0],x=results.iloc[:,0],c=\'deepskyblue\',label=\'Training\')\nplt.scatter(x,y, c=\'blue\',label=\'Test\')\nplt.plot([50,300],[50,300],label =\'True line\',c=\'red\')\nplt.legend(loc=\'lower right\')\n#plt.title(label=\'SVM-EC-Results-Plot\')\n\nplt.xlabel(r\'Experimental Discharge Capacity (Ah)\')\n\n#$50^{th}$ Cycle\nplt.ylabel(r\'Prediscted $50^{th}$ Cycle Discharge Capacity (mAh/g)\')\nplt.text(60,230,r\'$RMSE_{train}=Ah$\'+\'\n\'+r\'$RMSE_{test}=Ah$\' +"\n"+ \'$R^2_{test}=$\')\n\nplt.savefig(\'KNN-Predicted-vs-test.png\',dpi=600)\nplt.show()\n'